In [1]:
import pandas as pd
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
import nltk
from sklearn.linear_model import SGDClassifier
from konlpy.tag import Mecab
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from tensorflow import keras
mecab = Mecab()
#fm.findSystemFonts()
plt.rcParams['font.family']= ["NanumGothicCoding"]
plt.rcParams["axes.unicode_minus"]=False
# GPU 환경 설정하기
# assert tf.test.is_gpu_available() == True, 'GPU 설정을 확인하세요.'
print(tf.config.list_physical_devices('GPU'))
print(tf.config.list_logical_devices('GPU'))
print(tf.config.list_logical_devices('CPU'))

[]
[]
[LogicalDevice(name='/device:CPU:0', device_type='CPU')]


In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
import pandas as pd
import numpy as np
import urllib.request
import os
from tqdm import tqdm
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel

In [4]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [5]:
# 데이터를 가져옵니다.
df = pd.read_csv('spam.csv')
f = open("spam.csv", "r")
lines = f.read()

In [6]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
result = le.fit_transform(df['label'])
print(result)
df['label_num'] = result
df.head()
#spam이면 1, spam아니면 0
df.tail()

[1 1 1 ... 1 0 1]


,text,label,label_num
20095,"[Web발신]이상투자황기봉님""이재훈""투자컨설담당자배정완료.담당자확인▼http://b...",spam,1
20096,[Web발신](광고)조마?峙???충?矛?%:까지 ↓주소Joma.카JM톡무료거부,spam,1
20097,[Web발신]김환택님지금이라도늦지않았습니다.월긴급정보http://pf.kakao.c...,spam,1
20098,[Web발신]년노하우명품구매대행논스토리nonstory.com항상아낌없는사랑과성원에감...,ham,0
20099,[Web발신]최현님월일특급정보!https://pf.kakao.com/_SfcsT,spam,1


In [7]:
len(le.classes_)

2

In [8]:
df[:5]

,text,label,label_num
0,[Web발신]박춘규회원님손절주식은그만월급배만드는법http://lco.jp/eA,spam,1
1,[Web발신]골든브릿지에서손실을보셨나요??http://bitly.kr/bRGtq[FW],spam,1
2,[Web발신][대//박]가(.원)입코(WN)드MEP.com,spam,1
3,[Web발신](광고)이정미님아직도주식하시나요http://pf.kakao.com/_u...,spam,1
4,[Web발신]송승용님사람이모이는곳에는이유가있습니다하루~정보공개is.gd/JsJP,spam,1


In [9]:
df = df.dropna()

In [10]:
df['label'].value_counts()

spam    12817
ham      7272
Name: label, dtype: int64

In [11]:
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

In [12]:
df["text"] = df["text"].str.replace(pat=r'[^\w]', repl=r'', regex=True)

In [13]:
train_x = df['text'].to_list()
train_y = df['label_num'].to_list()

In [14]:
from sklearn.model_selection import train_test_split
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2, random_state=0)

In [15]:
train_x_encodings = tokenizer(train_x, truncation=True, padding = True)
val_x_encodings = tokenizer(val_x, truncation = True, padding = True)

In [16]:
nouns = mecab.nouns(lines)

In [17]:
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_x_encodings), train_y))
val_dataset = tf.data.Dataset.from_tensor_slices((dict(val_x_encodings), val_y))

In [18]:
from transformers import TFBertForSequenceClassification

In [19]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(patience = 5, verbose = 1, restore_best_weights= True)

In [22]:
num_labels = len(le.classes_)
model = TFBertForSequenceClassification.from_pretrained('klue/bert-base', num_labels = num_labels, from_pt = True)

model.compile(loss = model.compute_loss, optimizer = keras.optimizers.Adam(), metrics = ['accuracy'])
model.summary

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<bound method Model.summary of <transformers.models.bert.modeling_tf_bert.TFBertForSequenceClassification object at 0x7f54f50d5748>>

In [23]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  110617344 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 110,618,882
Trainable params: 110,618,882
Non-trainable params: 0
_________________________________________________________________


In [29]:
model.fit(train_dataset.shuffle(10).batch(16), validation_data = val_dataset.shuffle(10).batch(16), epochs = 4, batch_size = 16, callbacks = [es], verbose = 1)

Epoch 1/4
  32/1005 [..............................] - ETA: 3:27:13 - loss: 0.6910 - accuracy: 0.6250

In [25]:
from transformers import TFTrainer, TFTrainingArguments

training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=4,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs'            # directory for storing logs
)

with training_args.strategy.scope():
    trainer_model = TFBertForSequenceClassification.from_pretrained('klue/bert-base', num_labels=num_labels, from_pt=True)

trainer = TFTrainer(
    model=trainer_model,                 # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.6/dist-packages/transformers/trainer_tf.py:

In [28]:
trainer.train()

TypeError: '>' not supported between instances of 'NoneType' and 'int'